In [89]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import FloatSlider
import pandas as pd
import sklearn
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence
import numpy as np
import seaborn as sns
import shap
import matplotlib.pyplot as plt
import sklearn.preprocessing as skp
import pingouin as pg
from OLS_LR_DiagnosticPlots.ModelDiagnostics import Plot
import numpy as np
from scipy import linalg
import scipy
from sklearn.utils import check_array, as_float_array
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn import datasets
from bokeh.plotting import figure, show, output_notebook
from IPython.display import display, clear_output
from scipy.stats import moment
output_notebook()

Loading BokehJS ...

In [ ]:
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

def whiten(X, method='zca'):
		"""
		Whitens the input matrix X using specified whitening method.
		Inputs:
			X:      Input data matrix with data examples along the first dimension
			method: Whitening method. Must be one of 'zca', 'zca_cor', 'pca',
					'pca_cor', or 'cholesky'.
		"""
		X = X.reshape((-1, np.prod(X.shape[1:])))
		X_centered = X - np.mean(X, axis=0)
		Sigma = np.dot(X_centered.T, X_centered) / X_centered.shape[0]
		W = None
		if method in ['zca', 'pca', 'cholesky']:
			U, Lambda, _ = np.linalg.svd(Sigma)
			if method == 'zca':
				W = np.dot(U, np.dot(np.diag(1.0 / np.sqrt(Lambda + 1e-5)), U.T))
			elif method =='pca':
				W = np.dot(np.diag(1.0 / np.sqrt(Lambda + 1e-5)), U.T)
			elif method == 'cholesky':
				W = np.linalg.cholesky(np.dot(U, np.dot(np.diag(1.0 / (Lambda + 1e-5)), U.T))).T
		elif method in ['zca_cor', 'pca_cor']:
			V_sqrt = np.diag(np.std(X, axis=0))
			P = np.dot(np.dot(np.linalg.inv(V_sqrt), Sigma), np.linalg.inv(V_sqrt))
			G, Theta, _ = np.linalg.svd(P)
			if method == 'zca_cor':
				W = np.dot(np.dot(G, np.dot(np.diag(1.0 / np.sqrt(Theta + 1e-5)), G.T)), np.linalg.inv(V_sqrt))
			elif method == 'pca_cor':
				W = np.dot(np.dot(np.diag(1.0/np.sqrt(Theta + 1e-5)), G.T), np.linalg.inv(V_sqrt))
		else:
			raise Exception('Whitening method not found.')
		return np.dot(X_centered, W.T)

In [ ]:
df_ = pd.read_csv("https://raw.githubusercontent.com/thistleknot/Python-Stock/master/data/raw/states.csv") 

In [ ]:
df = sm.add_constant(df_)[[*df_.columns,'const']]

In [ ]:
#df[['Poverty', 'White', 'Traf Deaths', 'University', 'Unemployed', 'Income', 'Population']]

In [ ]:
'''
output_figure = widgets.Output()
 
# Create the default figure
fig = []  # Storing the figure in a singular list is a bit of a 
          # hack. We need it to properly mutate the current
          # figure in our callbacks.
#p = create_figure(
#    iris['feature_names'][0],
#    iris['feature_names'][1],
#    data)
#fig.append(p)
with output_figure:
    interact(derive_xnames,y=y)
    #interact(return_model_vars,x=x_,y=y,autoremove=autoremove)
    #show(fig)
    
app_layout = widgets.Layout(display='flex',
                flex_flow='row nowrap',
                align_items='center',
                border='none',
                width='100%',
                margin='5px 5px 5px 5px')
 
# The final app is just a box
app=widgets.Box([y, output_figure], layout=app_layout)
 
# Display the app
display(app)
'''

In [ ]:
def return_model_vars(x,y,autoremove):
    #print(x)
    names = [y,*x]
    print(names)
    
    shap.initjs()
    
    data = df[names].sort_values(kind="quicksort", by=names[0], ascending=True,key=abs).copy()
    
    loop=True
    
    removes = []
    if(autoremove):
        while(loop==True):

            x_names = data.columns[~data.columns.isin([item for item in [y,*removes]])]

            #x_scores = data[[y,*x_names]].pcorr()[[y]]

            x_scores = pd.concat([data[y],pd.DataFrame(whiten(np.array(data[x_names])),columns=x_names).set_index(data.index)],axis=1).pcorr()[[y]]

            #print(x_scores.loc[~x_scores.index.isin([y])])

            print(x_names)

            index_set = data.index

            test_results = x_scores

            n = len(index_set)
            df_ = n - 2

            t=abs(test_results)*np.sqrt(df_)/np.sqrt(1-abs(test_results)**2)

            test_results_least = t.iloc[[np.argmin(abs(t))]].index[0]

            t_score_least = t.iloc[[np.argmin(abs(t))]]

            crit_t = scipy.stats.t.ppf(1 - .05 / 2, df_)

            if(np.isnan(t_score_least.values[0][0])):
                remove = test_results_least
                removes.append(remove)

            if((t.loc[test_results_least]<crit_t)[0]):
                remove = test_results_least
                #print(remove,t_score_least.values[0][0])
                removes.append(remove)

            if((t.loc[test_results_least]>crit_t)[0]):
                loop=False

            if(len(x_names)==1):
                loop=False
    else:
        x_names = data.columns[~data.columns.isin([item for item in [y,*removes]])]

    return([y,*x_names])
    

In [ ]:
def return_model_subset(x,y,autoremove):
    #print(x)
    names = [y,*x]
    print(names)
    
    shap.initjs()
    
    data = df[names].sort_values(kind="quicksort", by=names[0], ascending=True,key=abs).copy()
    
    loop=True
    
    removes = []
    if(autoremove):
        while(loop==True):

            x_names = data.columns[~data.columns.isin([item for item in [y,*removes]])]

            #x_scores = data[[y,*x_names]].pcorr()[[y]]

            x_scores = pd.concat([data[y],pd.DataFrame(whiten(np.array(data[x_names])),columns=x_names).set_index(data.index)],axis=1).pcorr()[[y]]

            #print(x_scores.loc[~x_scores.index.isin([y])])

            print(x_names)

            index_set = data.index

            test_results = x_scores

            n = len(index_set)
            df_ = n - 2

            t=abs(test_results)*np.sqrt(df_)/np.sqrt(1-abs(test_results)**2)

            test_results_least = t.iloc[[np.argmin(abs(t))]].index[0]

            t_score_least = t.iloc[[np.argmin(abs(t))]]

            crit_t = scipy.stats.t.ppf(1 - .05 / 2, df_)

            if(np.isnan(t_score_least.values[0][0])):
                remove = test_results_least
                removes.append(remove)

            if((t.loc[test_results_least]<crit_t)[0]):
                remove = test_results_least
                #print(remove,t_score_least.values[0][0])
                removes.append(remove)

            if((t.loc[test_results_least]>crit_t)[0]):
                loop=False

            if(len(x_names)==1):
                loop=False
    else:
        x_names = data.columns[~data.columns.isin([item for item in [y,*removes]])]

    print(x_names)
    
    #X = data.iloc[:,1:]
    
    X = data[x_names]
   
    X_ = pd.DataFrame(whiten(np.array(X)),columns=X.columns).set_index(X.index)
    X_.index = X.index
    X_.columns = X.columns
    X = X_
    #print(X)
    #Y = scale(data.iloc[:,0], scale=True)
    Y = pd.DataFrame(skp.scale(data.iloc[:,0], with_mean=True, with_std=True))
    Y.columns = [y]
    Y.index = data.iloc[:,0].index
    #Y.sort_values(by=y,ascending=False,inplace=True)
    #X = X.loc[Y.index]
    
    model = sklearn.linear_model.LinearRegression()
    
    model.fit(X, Y)
    model_ = sm.OLS(Y,X)
    results = model_.fit()
    #shap
    background = np.array(X)
    e = shap.LinearExplainer(model, X)
    
    shap_values = e.shap_values(np.array(X))
    shap.summary_plot(shap_values, -np.array(X))
    explainer = shap.Explainer(model, X)
    shap.plots.heatmap(explainer(X))
    
    predict = results.predict(X.loc[Y.index])
    plt.plot(np.array(Y))
    plt.plot(np.array(predict))
    plt.plot(np.array(predict.rolling(3).mean()))
    plt.show()
    
    corrMatrix = pd.concat([Y,X],axis=1).corr().sort_values(kind="quicksort", by=names[0], ascending=True,key=abs)
    sns.heatmap(corrMatrix, annot=True)
    plt.show()
    
    corrMatrix = pd.concat([Y,X],axis=1).pcorr().sort_values(kind="quicksort", by=names[0], ascending=True,key=abs)
    sns.heatmap(corrMatrix, annot=True)
    plt.show()
    
    (predict-Y[y]).hist()
    plt.show()
    
    print("mape:",MAPE(Y[y],predict))
    
    linear_plot = Plot.LinearRegressionResidualPlot(X.values, Y.values)
    lm = linear_plot.fit()
    summary, diag_res = linear_plot.diagnostic_plots(lm)
    #print("Summary of Regression\n:{}".format(summary))
    print("Diagnostic Tests of Regression\n:{}".format(diag_res))
    sns.set_theme(style="ticks")
    
    temp = pd.concat([Y,X],axis=1)
    temp.index = df['States'].loc[X.index]
    #temp['target']=temp[y]
    sns.pairplot(temp,hue=y)
    #return([results.summary(),temp])
    return([results.summary()])

def derive_xnames(y):
    x_ = widgets.SelectMultiple(
        options=df.columns[~df.columns.isin([y])][1:],
        value=list(df.columns[~df.columns.isin([y])][1:]),
        disabled=False
    )
    autoremove = widgets.Checkbox()
    subset = interact(return_model_subset,x=x_,y=y,autoremove=True)
    #subset = interact(return_model_vars,x=x_,y=y,autoremove=autoremove)

    return(subset)

y=widgets.Select(options=df.columns[1:-1],disabled=False)

a=interact(derive_xnames,y=y)

